# Webscraping Bank Product Reviews
BoA Datasource = https://wallethub.com/profile/bank-of-america-13000450i

In [30]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import  Options
import pandas as pd
import time
import datetime
from random import uniform

In [31]:
# Use Driver to fetch website
url = "https://wallethub.com/profile/bank-of-america-13000450i" 
DRIVER_PATH = "/Users/jesidacosta/OneDrive - University of South Florida/ISM6930/group_project/chromedriver"
# Create Headless Session (not GUI)
options = Options()
options.headless = True
# Create Driver
driver = webdriver.Chrome(DRIVER_PATH, options=options)
# driver.get(url + "?p=" + "3")

# Fetch HTML using Beautiful Soup
# time.sleep(1)
# soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')

# for review in soup.find_all(class_='rvtab-citem'):
#     author_name = review.find(class_='rvtab-ci-name').text.strip()
#     date = review.find('time')['datetime']
#     comment = review.find(class_='rvtab-ci-content').text
#     print(date, author_name, comment, '\n')

In [32]:
def get_html_file(url, driver, page=None):
    if page != None:
        url += "?p=" + page
    # inserting random time lapse to try to prevent bot detection
    time.sleep(uniform(1.0,3.0)) 
    # fetch website
    driver.get(url)
    # get html
    soup = BeautifulSoup(driver.page_source, 'lxml', from_encoding='UTF-8')
    return soup

In [33]:
html = get_html_file(url, driver)
page_position = html.find(class_='rvtab-pag-pos').text
current_page = page_position.split()[0]
last_page = page_position.split()[2]

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
print(current_page, last_page)

1 284


In [ ]:
# Create DataFrame for storage
col_names = ['date', 'name', 'user_id', 'verified', 'product', 'review_comment', 'stars', 'review']
data = pd.DataFrame(columns=col_names)

In [ ]:
def fetch_page_reviews(html, df):
    for review in html.find_all(class_='rvtab-citem'):  
        # Initialize Values
        stars = 0
        verified = True
        review_comment = False
        # Name & UserName
        author_name = review.find(class_='rvtab-ci-name').text.strip()
        nickname = review.find(class_='rvtab-ci-nickname').text[1:]
        # Date of Review
        date = review.find('time')['datetime']
        # Product Reviewed
        if review.find(class_='rvtab-ci-category') is not None:
            product = review.find(class_='rvtab-ci-category').text.replace('Product:','').strip()
        else:
            # prints comments that don't belong to specific product, probably comments to previous review
            print(author_name, nickname, date, verified, stars)
            review_comment = True
            product = None
        # Review Comment
        comment = review.find(class_='rvtab-ci-content').text
        # Count number of stars
        for star in review.find_all('path'):
            if star['fill'] == '#4ae0e1':
                stars += 1
        if review.find(class_='rvtab-ci-verified') is None:
            verified = False
            # print(author_name, nickname, date, verified, stars)
            # print(product)
            # print('\t', comment)
        # temporary to add row by row
        row = [[date, author_name, nickname, verified, product, review_comment, stars, comment]]
        temp = pd.DataFrame(row, columns=df.columns)
        # add to main dataframe
        df = df.append(temp, ignore_index=True)
    return df

In [ ]:
data = fetch_page_reviews(html, data)
data

,date,name,user_id,verified,product,stars,comment
0,2021-09-04,tomalbert848,tomalbert848,False,Bank of America Credit Cards,5,"I had some nagging late payments, medical bill..."
1,2021-09-04,Hobed,hobed,True,Bank of America Credit Cards,3,"Overall, the experience has been mostly positi..."
2,2021-09-04,totoalbert867,totoalbert867,False,Bank of America Credit Cards,5,I just feel like crying. I was back and forth ...
3,2021-09-04,ctiwan12,ctiwan12,False,Bank of America Credit Cards,5,I want a new open account BOA but I have a bad...
4,2021-09-03,scottcole914,scottcole914,False,Bank of America Car Loans,5,"Congratulation on the new Location, I rate 5 s..."
5,2021-09-02,Jonathon,jonathon_lr_bell,True,Bank of America Credit Cards,3,Nice low rate and long 0 interest entry promot...
6,2021-09-01,Patricia,patriciad_145,True,Bank of America Credit Cards,5,Have been with BOA for about 15 years now. Geo...
7,2021-08-31,Richard,moredock,True,Bank of America Credit Cards,4,The card has had a relatively low interest rat...
8,2021-08-29,Mohammed,mohammeda_10,True,Bank of America Credit Cards,5,Bank of America credit card sucks I wouldn’t r...
9,2021-08-29,sonjapenner,sonjapenner,False,Bank of America Credit Cards,1,They started out great but I recently have bee...


In [ ]:
# for review in html.find_all(class_='rvtab-citem'):  
#     # Initialize Values
#     stars = 0
#     verified = True
#     # Name & UserName
#     author_name = review.find(class_='rvtab-ci-name').text.strip()
#     nickname = review.find(class_='rvtab-ci-nickname').text[1:]
#     # Date of Review
#     date = review.find('time')['datetime']
#     # Product Reviewed
#     product = review.find(class_='rvtab-ci-category').text.replace('Product:','').strip()
#     # Review Comment
#     comment = review.find(class_='rvtab-ci-content').text
#     # Count number of stars
#     for star in review.find_all('path'):
#         if star['fill'] == '#4ae0e1':
#             stars += 1
#     if review.find(class_='rvtab-ci-verified') is None:
#         verified = False
#         # print(author_name, nickname, date, verified, stars)
#         # print(product)
#         # print('\t', comment)
#     # temporary to add row by row
#     row = [[date, author_name, nickname, verified, product, stars, comment]]
#     temp = pd.DataFrame(row, columns=col_names)
#     # add to main dataframe
#     data = data.append(temp, ignore_index=True)
# data

In [ ]:
def scrape_reviews(url, driver):
    # Create DataFrame for storage
    col_names = ['date', 'name', 'user_id', 'verified', 'product', 'stars', 'comment']
    data = pd.DataFrame(columns=col_names)
    # fetch starting page
    html = get_html_file(url, driver)
    # get page positions
    page_position = html.find(class_='rvtab-pag-pos').text
    current_page = page_position.split()[0]
    last_page = page_position.split()[2]
    while int(current_page) <= int(last_page):
        print(current_page)
        html = get_html_file(url, driver, current_page)
        # check comment review exists in page - else bot may be blocked
        if html.find_all(class_='rvtab-citem'):
            # add page reviews
            data = fetch_page_reviews(html, data)
            # get new page number
            current_page = html.find(class_='rvtab-pag-pos').text.split()[0]
            # increase page
            current_page = str(int(current_page) + 1)
        elif html.title.text == 'IP Block':
            print('No more comments retrieved - Bot Blocked')
            break
    driver.close()
    return data

In [ ]:
data = scrape_reviews(url,driver)
data

/Users/jesidacosta/.local/share/virtualenvs/ISM6930-CY7GBNHA/lib/python3.9/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
Angel Naranjo naran1 2021-04-03 True 0
23
24
25
26
27
28
vietnamkindergarten vietnamkindergarten 2021-03-01 True 0
29
30
31
elenascott1224 elenascott1224 2021-02-03 True 0
elenascott1224 elenascott1224 2021-02-03 True 0
cheryll428 cheryll428 2021-02-03 True 0
32
cheryll428 cheryll428 2021-02-03 True 0
MICHAEL michaelb_1514 2021-02-01 True 0
Peter Tran peter_tran_cm 2021-01-28 True 0
33
34
35
Mildred Klemp HarleyClark 2021-01-08 True 0
36
purchasing purchasing 2021-01-06 True 0
37
38
39
40
ayowolej ayowolej 2020-12-16 True 0
ayowolej ayowolej 2020-12-16 True 0
41
ayowolej ayowolej 2020-12-16 True 0
ayowolej ayowolej 2020-12-16 True 0
ayowolej ayowolej 2020-12-16 True 0
ayowolej ayowolej 2020-12-16 True 0
coreyrobinson coreyrobinson 2021-05-08 True 0
42
43
44
45
broderick_a_hall broderick_a_hall 2020-11-14 True 0
46
47
jinweisiger jinweisiger 2020-10-31 True 0
48
49
Jennifer Click jengagers 2020-10-20 True 0
Jennifer Click jengager

,date,name,user_id,verified,product,stars,comment
0,2021-09-04,tomalbert848,tomalbert848,False,Bank of America Credit Cards,5,"I had some nagging late payments, medical bill..."
1,2021-09-04,Hobed,hobed,True,Bank of America Credit Cards,3,"Overall, the experience has been mostly positi..."
2,2021-09-04,totoalbert867,totoalbert867,False,Bank of America Credit Cards,5,I just feel like crying. I was back and forth ...
3,2021-09-04,ctiwan12,ctiwan12,False,Bank of America Credit Cards,5,I want a new open account BOA but I have a bad...
4,2021-09-03,scottcole914,scottcole914,False,Bank of America Car Loans,5,"Congratulation on the new Location, I rate 5 s..."
...,...,...,...,...,...,...,...
1660,2014-09-08,Jill G,lambchop522,False,Bank of America Checking,2,I used this checking account only for a short ...
1661,2014-09-04,McKenzie Thompson,mckenzie_thompson_9,False,Bank of America Credit Cards,5,"This has been a good card for me, although the..."
1662,2014-09-03,Laurel Stuart,laurel_artemis_stuart,False,Bank of America Checking,4,I have been a Bank of America checking custome...
1663,2014-09-03,Mary Leo,vero_leal_357,False,Bank of America Checking,5,I have had many bank accounts through out the ...


In [36]:
# Save Document as Pandas
DATE = datetime.datetime.now().date()
data.to_pickle("boa_wallethub_reviews_{}.pkl".format(DATE))
data.to_csv("boa_wallethub_reviews_{}.csv".format(DATE), index=False, encoding='utf-8')

In [37]:
# Encoding seems to be a problem in excel not in 
data.iloc[26,:].comment

'I make payments on time I missed one payment and BoA did not waive my late fees. WORST CUSTOMER SERVICE EVER! Very rude reps don’t even want to listen to you. I have been a loyal customer for years and I forgot to make a payment by 2 days and it was my first time getting late fee they did not even consider to waive it off!!!'

In [43]:
df = pd.read_pickle('boa_wallethub_reviews_2021-09-05.pkl')

In [45]:
df.iloc[22].comment

'The card has a $39 per year fee. I’ve had it almost 7 years still a low limit - $2000. My other cards have MUCH higher limits. I have great credit.  I keep it to use for internet purchases, it’s easily monitored and keeps my other higher limit cards safe from the web.'